#### Imports

In [14]:
from bs4 import BeautifulSoup
import requests
import re
import json
from newspaper import Article
from datetime import datetime

In [116]:
members = ['mario draghi', 'vitor constancio', 'benoit coeure', 'yves mersche', 'sabine lautenschlager', 'peter praet']
# representatives = ['luc coene', 'jens weidmann', 'patrick honohan', 'yannis stournaras', 'lui maria linde', 'ardo hansson',
#                    'christian noyer', 'ignazio visco', ]

#### Grab homepages, search from there
start:
- project Syndicate
- WorldCrunch
- Eurozine

In [39]:
sites = ['https://www.project-syndicate.org/', 'http://www.worldcrunch.com/business-finance', 'http://www.eurozine.com/']
r = requests.get(sites[1])
worldcrunch_soup = BeautifulSoup(r.text, 'lxml')

In [40]:
links = worldcrunch_soup.find_all('a')

In [41]:
title = re.compile('[A-Z][\w-]')

article_links = []

no_match = 0

for link in links:
    if title.match(link.text.strip()):
        article_links.append(link)
    else:
        no_match += 1
        
print('Number of links : {}'.format(str(len(article_links))))
print('Other links: {}'.format(str(no_match)))
        

Number of links : 50
Other links: 86


## NOTE: have to do this multiple times to get to 29 links

In [44]:
# stops = ['SUBSCRIBE', 'subscribe', 'Terms of Use', 'About Us', 'Privacy Policy']

for link in article_links:
    if len(link.text.split()) < 3:
        article_links.remove(link)
    elif '#' in str(link):
        article_links.remove(link)

print(len(article_links))

29


#### use newspaper to extract articles from remaining links

In [46]:
base_url = 'http://www.worldcrunch.com/'

def parseArticle(base_url, article_links):
    
    worldcrunch_dic = {}

    for link in article_links[2:]:
        url = base_url+link['href']
        r = requests.get(url)
        # Beautiful Soup
        soup = BeautifulSoup(r.text, 'lxml')

        try:
            publish_date = soup.find("meta", {"name":"DISPLAYDATE"})['content']
            r_datetime = datetime.strptime(publish_date, '%Y-%m-%d %H:%M:%S')
            d = datetime.strftime(r_datetime, '%Y-%m-%d')
            t = datetime.strftime(r_datetime, '%H:%M:%S')
            author = soup.find_all("p", {"class":"resize-auth"})[0].find('a').text.strip()
            raw_text = soup.find("div", {"class":"article-text"}).text
            if d in worldcrunch_dic:
                worldcrunch_dic[d].append([author, t, raw_text])
            else:
                worldcrunch_dic[d] = [(author, t, raw_text)]
        except:
            pass
    return worldcrunch_dic

worldcrunch_dic = parseArticle(base_url, article_links)

In [48]:

with open('worldcrunch.json', 'w') as fp:
    json.dump(worldcrunch_dic, fp)

In [50]:
len(worldcrunch_dic.keys())

22

#### Try methods for syndicate

In [192]:
search_url = 'https://www.project-syndicate.org/search?query=mario%20draghi&language=english&sortBy=publishedon&sortOrder=desc'
base_url = 'https://www.project-syndicate.org'

In [177]:
r = requests.get(search_url)
soup = BeautifulSoup(r.text, 'lxml')
article_links = soup.find_all("section", {"class":"small-12 medium-12 large-7 columns search-results listing authors"})[0].find_all('article', {"class":"section economics "})

In [77]:
def NewspaperParse(url, authors='authors'):
    article = Article(url)   
    article.download()
    article.parse()
    r_datetime = article.publish_date
    d = datetime.strftime(r_datetime, '%Y-%m-%d')
    t = datetime.strftime(r_datetime, '%H:%M:%S')
    raw_text = article.text
    if authors =='authors':
        author = article.authors[0]
    else:
        author = 'none'
    
    return d, t, raw_text, author 

def PS_parse(base_url, article_links):
    ps_dic = {}
    for link in article_links:
        ext = link.find("h2").find('a')['href']
        url = base_url + ext
        dt, raw_text, author = NewspaperParse(url)
        ps_dic[dt] = (author, raw_text)
    return ps_dic

# ps_dic = PS_parse(base_url, article_links)

In [310]:
with open('ps.json', 'w') as fp:
    json.dump(ps_dic, fp)

### BBC News

In [231]:
search_url = 'http://www.bbc.com/news/business'
base_url ='http://www.bbc.com'

r = requests.get(search_url)
soup = BeautifulSoup(r.text, 'lxml')

title_ext = soup.find('div',{"class":"column--primary"}).find('a', {"class":"title-link"})['href']


'/news/business-38507368'

In [17]:
a = Article('http://www.bbc.com/news/business-38507368')
a.download()
a.parse()
a.text

'Image copyright Getty Images\n\nEurozone inflation has surged to its highest rate in more than three years, driven by increased prices for energy, food, alcohol and tobacco.\n\nThe annual inflation rate hit 1.1% last month, according to official statistics agency Eurostat, a sharp jump from November\'s rate of 0.6%.\n\nThe rate is the highest since September 2013, when inflation was also 1.1%.\n\nThe higher-than-expected increase brings inflation closer to the European Central Bank\'s target of just below 2%.\n\nECB chief Mario Draghi has said he expects inflation to reach the target by 2018 or 2019.\n\nLast month\'s increase was driven mainly by a jump in energy prices, which rose by 2.5% year-on-year in December, their first increase in over a year. Energy prices were boosted by oil cartel Opec\'s decision to cut output.\n\nFood, alcohol and tobacco prices rose 1.2% year-on-year, while services were also 1.2% more expensive than a year ago.\n\nShort-lived rise?\n\nThe rise will help

In [20]:
r = requests.get('http://www.bbc.com/news/business-38507368').text

In [21]:
soup = BeautifulSoup(r, 'lxml')

In [23]:
print(soup.prettify())

<!DOCTYPE html>
<html id="responsive-news" lang="en">
 <head prefix="og: http://ogp.me/ns#">
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <title>
   Eurozone inflation surges to 1.1% in December - BBC News
  </title>
  <meta content="Eurozone inflation surges to 1.1%, marking its highest rate in more than three years." name="description"/>
  <link href="https://ssl.bbc.co.uk/" rel="dns-prefetch"/>
  <link href="http://sa.bbc.co.uk/" rel="dns-prefetch"/>
  <link href="http://ichef-1.bbci.co.uk/" rel="dns-prefetch"/>
  <link href="http://ichef.bbci.co.uk/" rel="dns-prefetch"/>
  <link href="//c.go-mpulse.net/" rel="dns-prefetch"/>
  <meta content="nl" name="x-country"/>
  <meta content="International" name="x-audience"/>
  <meta content="International" name="CPS_AUDIENCE"/>
  <meta content="295110099" name="CPS_CHANGEQUEUEID"/>
  <link href="http://www.bbc.com/news/business-38507368" rel="canonical"/>
  <link href="http://www.bbc.co.uk/new

In [35]:
date = datetime.strptime(soup.find("div", {"class":"date date--v2"})['data-datetime'], '%d %B %Y')

datetime.datetime(2017, 1, 4, 0, 0)

In [270]:
# search_url = 'http://www.bbc.co.uk/search?q=mario+draghi#page=20'
result_list = []
for i in range(1,20):
    search_url = 'http://www.bbc.co.uk/search?q=mario+draghi#page={}'.format(str(i))
    print(search_url)
    r = requests.get(search_url)
    print(r.status_code)
    soup = BeautifulSoup(r.text, 'lxml')
    results = soup.find_all("ol", {"class":"search-results results"})
    r_links = [[result['href'] for result in rs.find_all('a')] for rs in results]
    result_list.extend(r_links)

http://www.bbc.co.uk/search?q=mario+draghi#page=1
200
http://www.bbc.co.uk/search?q=mario+draghi#page=2
200
http://www.bbc.co.uk/search?q=mario+draghi#page=3
200
http://www.bbc.co.uk/search?q=mario+draghi#page=4
200
http://www.bbc.co.uk/search?q=mario+draghi#page=5
200
http://www.bbc.co.uk/search?q=mario+draghi#page=6
200
http://www.bbc.co.uk/search?q=mario+draghi#page=7
200
http://www.bbc.co.uk/search?q=mario+draghi#page=8
200
http://www.bbc.co.uk/search?q=mario+draghi#page=9
200
http://www.bbc.co.uk/search?q=mario+draghi#page=10
200
http://www.bbc.co.uk/search?q=mario+draghi#page=11
200
http://www.bbc.co.uk/search?q=mario+draghi#page=12
200
http://www.bbc.co.uk/search?q=mario+draghi#page=13
200
http://www.bbc.co.uk/search?q=mario+draghi#page=14
200
http://www.bbc.co.uk/search?q=mario+draghi#page=15
200
http://www.bbc.co.uk/search?q=mario+draghi#page=16
200
http://www.bbc.co.uk/search?q=mario+draghi#page=17
200
http://www.bbc.co.uk/search?q=mario+draghi#page=18
200
http://www.bbc.co.u

#### SELENIUM, return later

In [2]:
# from selenium.remote.webdriver import WebDriver

In [206]:
from selenium.webdriver.common.keys import Keys

In [225]:
import time
import selenium.webdriver.chrome.service as service
driver = webdriver.Chrome('/usr/local/chromedriver')

In [226]:
driver.get('https://www.google.co.uk/')

In [227]:
time.sleep(5)
search_box = driver.find_element_by_name('q')
search_box.send_keys('ChromeDriver')
search_box.submit()
time.sleep(5) # Let the user actually see something!
driver.quit()

### Gaurdian


In [37]:
import json

with open('../data/guardianresults.json') as data_file:    
    guardian_results = json.load(data_file)

In [38]:
guardian_results

{'response': {'currentPage': 1,
  'orderBy': 'relevance',
  'pageSize': 161,
  'pages': 1,
  'results': [{'apiUrl': 'https://content.guardianapis.com/business/2015/jan/22/european-union-stimulus-gains-us-stock-markets-gold-price-dollar',
    'id': 'business/2015/jan/22/european-union-stimulus-gains-us-stock-markets-gold-price-dollar',
    'isHosted': False,
    'sectionId': 'business',
    'sectionName': 'Business',
    'type': 'article',
    'webPublicationDate': '2015-01-22T20:30:08Z',
    'webTitle': "Who loves Europe's bold €1.1tn financial stimulus? Americans",
    'webUrl': 'https://www.theguardian.com/business/2015/jan/22/european-union-stimulus-gains-us-stock-markets-gold-price-dollar'},
   {'apiUrl': 'https://content.guardianapis.com/business/2014/apr/03/european-central-bank-meets-as-deflation-fears-grow-business-live',
    'id': 'business/2014/apr/03/european-central-bank-meets-as-deflation-fears-grow-business-live',
    'isHosted': False,
    'sectionId': 'business',
    's

In [303]:
guardian_dic = {}

for result in guardian_results['response']['results']:
    if result['type'] == 'article':
        url = result['webUrl']
        
        article = Article(url)
        article.download()
        article.parse()
        raw_text = article.text
        author = article.authors
        r_datetime = article.publish_date
        d = datetime.strftime(r_datetime, '%Y-%m-%d')
        t = datetime.strftime(r_datetime, '%H:%M:%S')
        if d in guardian_dic:
            guardian_dic[d].extend([(author, t, raw_text)])
        else:
            guardian_dic[d] = [(author, t, raw_text)]
        

In [307]:
guardian_dic[list(guardian_dic.keys())[3]]

[(['Steve Boxer'],
  '00:00:00',
  'We can assume that David Cameron is a bit of a gamer, having been busted obsessively playing Fruit Ninja and Angry Birds in the past. So there is an outside chance that, finally, he might soften his government’s harsh, grudging response to the Syrian refugee crisis. Because one of the gaming world’s most famous characters – the moustachioed plumber Mario – has been co-opted into highlighting the plight of those attempting to flee from the Scylla and Charybdis of Isis and Bashar al-Assad.\n\nA Syrian artist, pseudonymously known as Samir al-Mutfi, has created Syrian Super Mario, a satirical video that is currently going viral, highlighting the horrors faced by Syrian refugees – by tweaking the 1985 video game Super Mario Bros. Mario has taken many forms over the decades, but this time he has become Refugee Mario. And instead of making his way to Bowser’s castle in order to rescue Princess Peach, he merely has to get to the castle, which now bears the 

In [309]:
with open('guardian.json', 'w') as fp:
    json.dump(guardian_dic, fp)

### The FT - need to get corporate developer license

In [6]:
# search_url = 'https://www.ft.com/search?q=mario+draghi'

# r = requests.get(search_url)
# soup = BeautifulSoup(r.text, 'lxml')

for li in soup.find_all('li', {"class": "o-teaser-collection__item o-grid-row"}):
    article_links.extend(li.find_all('a'))
    
    

In [22]:
link = article_links[1]['href']

In [13]:
base_url = 'http://ft.com' 

In [ ]:
def ParseArticles(base_url, article_links):
    for link in article_links:
        url = base_url + link
        r = requests.get(url)

In [23]:
url = base_url + link

In [34]:
article = Article(url)

In [35]:
article.download()
article.parse()
article.publish_date

In [36]:
article.text

'Subscribe to FT.com\n\nSubscribe to read: Italy’s president picks up pieces after political crisis\n\nAlready a subscriber? Sign in here'

### EurActiv - has no authors on articles

In [90]:
soup = BeautifulSoup(requests.get('http://www.euractiv.com/?s=mario+draghi').text)

/Users/hannahdeen/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [97]:
block = soup.find_all("div",{"class":"row"})
# article_links = []
    
# for b in block.find_all("a"):
#     if len(b['href'].split('/')) > 6 and b['href'] not in article_links:
#         article_links.append(b['href'])

In [101]:
block[4]

<div class="row ea-columns-same-height">
<div class="ea-archive col-xs-12 col-sm-12 col-md-12 col-lg-12">
<div class="ea-color-white"><br/><div class="row"><div class="col-xs-12 col-sm-6 col-md-4 col-lg-3">
<div class="ea-teaser ea-teaser-news">
<div class="ea-teaser-labels"><a href="http://www.euractiv.com/news/">News</a></div>
<a href="http://www.euractiv.com/section/euro-finance/news/draghi-trump-and-brexit-causing-economic-uncertainty/"><span class="image-background"><img class="lazy-load img-responsive hidden-no-js" data-src="http://www.euractiv.com/wp-content/uploads/sites/2/2016/04/European_Parliament_48141299.jpg" data-strategy="crop" src="//assets.euractiv.com/lazy-load/placeholder/16x9.png" style="width:100%"/><noscript><img class="img-responsive" src="http://www.euractiv.com/wp-content/uploads/sites/2/2016/04/European_Parliament_48141299.jpg" style="width:100%"/></noscript></span></a>
<div class="ea-teaser-meta"><a href="http://www.euractiv.com/sections/euro-finance/">Euro &

In [112]:
search_url = 'http://www.euractiv.com/?s=mario+draghi'
# 'http://www.euractiv.com/page/2/?s=mario+draghi'
# for all names, and 10 pages?

def getLinksEurActiv(search_url):
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'lxml')
    
    block = soup.find_all("div",{"class":"row"})[4]
    article_links = []
    
    for b in block.find_all("a"):
        if len(b['href'].split('/')) > 6 and b['href'] not in article_links:
            article_links.append(b['href'])
    
    return article_links


def NewspaperParse(url, authors='authors'):
    article = Article(url)   
    
    try:
        article.download()
        article.parse()
        r_datetime = article.publish_date
        d = datetime.strftime(r_datetime, '%Y-%m-%d')
        t = datetime.strftime(r_datetime, '%H:%M:%S')
        raw_text = article.text
        if authors =='authors':
            author = article.authors[0]
        else:
            author = 'none'
        return d, t, raw_text, author
    except:
        return 'none', 'none', 'none', 'none'
        

article_links = []
euractiv_dic = {}

for i in range(2,10):
    search_url = 'http://www.euractiv.com/page/{}/?s=mario+draghi'.format(i)
    print(search_url)
    links = getLinksEurActiv(search_url)
    article_links.extend(links)
    article_links = list(set(article_links))
    
    for link in article_links:
        d, t, raw_text, author = NewspaperParse(link, 'none')
        if d != 'none':
            if d in euractiv_dic:
                euractiv_dic[d].append([(author, t, raw_text)])
            else:
                euractiv_dic[d] = [(author, t, raw_text)]
    



http://www.euractiv.com/page/2/?s=mario+draghi
http://www.euractiv.com/page/3/?s=mario+draghi
http://www.euractiv.com/page/4/?s=mario+draghi
http://www.euractiv.com/page/5/?s=mario+draghi
http://www.euractiv.com/page/6/?s=mario+draghi
http://www.euractiv.com/page/7/?s=mario+draghi
http://www.euractiv.com/page/8/?s=mario+draghi
You must `download()` an article before calling `parse()` on it!
http://www.euractiv.com/page/9/?s=mario+draghi
You must `download()` an article before calling `parse()` on it!


In [118]:
'http://content.guardianapis.com/search?section=business&from-date=2001-01-01&page-size=161&q=vitor%20constancio&api-key=458f61b9-2ff0-4a3f-98a1-feff65660ca6'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31656  100 31656    0     0  38823      0 --:--:-- --:--:-- --:--:-- 38794


In [11]:
import requests
r = requests.get('http://content.guardianapis.com/search?section=business&from-date=2001-01-01&page-size=161&q=vitor%20constancio&api-key=458f61b9-2ff0-4a3f-98a1-feff65660ca6')

In [12]:
r.text

'{"response":{"status":"ok","userTier":"developer","total":60,"startIndex":1,"pageSize":161,"currentPage":1,"pages":1,"orderBy":"relevance","results":[{"id":"business/2015/jan/22/european-union-stimulus-gains-us-stock-markets-gold-price-dollar","type":"article","sectionId":"business","sectionName":"Business","webPublicationDate":"2015-01-22T20:30:08Z","webTitle":"Who loves Europe\'s bold €1.1tn financial stimulus? Americans","webUrl":"https://www.theguardian.com/business/2015/jan/22/european-union-stimulus-gains-us-stock-markets-gold-price-dollar","apiUrl":"https://content.guardianapis.com/business/2015/jan/22/european-union-stimulus-gains-us-stock-markets-gold-price-dollar","isHosted":false},{"id":"business/2014/apr/03/european-central-bank-meets-as-deflation-fears-grow-business-live","type":"liveblog","sectionId":"business","sectionName":"Business","webPublicationDate":"2014-04-03T16:42:11Z","webTitle":"Euro slides after ECB \'talks about\' QE to battle stagnation","webUrl":"https://

In [1]:
import json

with open('../data/guardian.json') as data_file:    
    guardian_results = json.load(data_file)

In [2]:
text = guardian_results['2016-08-05'][0][2]

In [6]:
# from google.cloud import language

# language_client = language.Client()
# document = language_client.document_from_text(text)
# sentiment = document.analyze_sentiment()
# entities = document.analyze_entities()

print('Score: {}'.format(sentiment.score))
print('Magnitude: {}'.format(sentiment.magnitude))

Score: 0.1
Magnitude: 5.8


In [5]:
for entity in entities:
        print('=' * 20)
        print('{:<16}: {}'.format('name', entity.name))
        print('{:<16}: {}'.format('type', entity.entity_type))
        print('{:<16}: {}'.format('wikipedia_url', entity.wikipedia_url))
        print('{:<16}: {}'.format('metadata', entity.metadata))
        print('{:<16}: {}'.format('salience', entity.salience))

name            : Arsène Wenger
type            : PERSON
wikipedia_url   : http://en.wikipedia.org/wiki/Ars%C3%A8ne_Wenger
metadata        : {'mid': '/m/01gjbw'}
salience        : 0.14021197
name            : picture
type            : WORK_OF_ART
wikipedia_url   : None
metadata        : {}
salience        : 0.10435785
name            : businessman
type            : PERSON
wikipedia_url   : None
metadata        : {}
salience        : 0.035288125
name            : bed
type            : OTHER
wikipedia_url   : None
metadata        : {}
salience        : 0.035288125
name            : transfer talk
type            : OTHER
wikipedia_url   : None
metadata        : {}
salience        : 0.035288125
name            : tittle-tattle
type            : EVENT
wikipedia_url   : None
metadata        : {}
salience        : 0.035288125
name            : Real Madrid
type            : ORGANIZATION
wikipedia_url   : http://en.wikipedia.org/wiki/Real_Madrid_C.F.
metadata        : {'mid': '/m/06l22'}
salience

In [7]:
from google.cloud import storage

In [8]:
bucket_name = 'dove-hawk-data'
source_file_name = '../data/guardian.json'
destination_blob_name = 'testblob'

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_blob_name))

In [9]:
upload_blob(bucket_name, source_file_name, destination_blob_name)

File ../data/guardian.json uploaded to testblob.
